In [3]:
from glob import glob
import pandas as pd

In [8]:
contribs = pd.read_csv("raw_contributions/COS2.csv")
party_lookup = {
    "D)": "democrat",
    "R)": "republican"
}
def extract_contrib(row):
    firstname = row["FirstLastP"].split(" ")[0].lower()
    lastname = row["FirstLastP"].split(" ")[1].lower()
    party = row["FirstLastP"].split("(")[1]
    party = party_lookup.get(party, "other")
    amount = row["Total"]
    contributor = row["Contrib"]
    return pd.Series({
        "firstname": firstname,
        "lastname": lastname,
        "party": party,
        "amount": amount,
        "contributor": contributor
        })
processed_contribs = contribs.apply(extract_contrib, axis=1)
file = "raw_contributions/COS2.csv"
race = file.split("/")[-1].split(".")[0]
processed_contribs = processed_contribs.assign(race=race)

,firstname,lastname,party,amount,contributor,race
0,cory,gardner,republican,124600,Votesane PAC,COS2
1,cory,gardner,republican,64961,Apollo Global Management,COS2
2,cory,gardner,republican,55979,GEO Group,COS2
3,cory,gardner,republican,51765,Blue Cross/Blue Shield,COS2
4,cory,gardner,republican,50788,Blackstone Group,COS2
5,cory,gardner,republican,44700,Elliott Management,COS2
6,cory,gardner,republican,42666,BGR Group,COS2
7,cory,gardner,republican,34554,NextEra Energy,COS2
8,cory,gardner,republican,34413,"Brownstein, Hyatt et al",COS2
9,cory,gardner,republican,34275,"Paul, Weiss et al",COS2


In [40]:
contribs

,affiliateid,orgid,pacid,cycle,CID,FirstLastP,Contrib,Indivs,Pacs,Total,Rank,CRPICO,ReleaseDate,NetRcpts
0,NaN,D000000113,NaN,2022,N00038397,Maggie Hassan (D),EMILY's List,428722,10000,438722,1,I,2022-10-28 00:00:00 +0000,38340375.03
1,NaN,D000025328,NaN,2022,N00038397,Maggie Hassan (D),Democracy Engine,406471,0,406471,2,I,2022-10-28 00:00:00 +0000,38340375.03
2,NaN,D000052457,NaN,2022,N00038397,Maggie Hassan (D),J Street,135999,0,135999,3,I,2022-10-28 00:00:00 +0000,38340375.03
3,NaN,D000065181,NaN,2022,N00038397,Maggie Hassan (D),Votesane PAC,104000,0,104000,4,I,2022-10-28 00:00:00 +0000,38340375.03
4,NaN,D000067823,NaN,2022,N00038397,Maggie Hassan (D),Alphabet Inc,88779,5000,93779,5,I,2022-10-28 00:00:00 +0000,38340375.03
5,NaN,D000000544,NaN,2022,N00038397,Maggie Hassan (D),Harvard University,91544,0,91544,6,I,2022-10-28 00:00:00 +0000,38340375.03
6,NaN,D000000406,NaN,2022,N00038397,Maggie Hassan (D),University of California,71585,0,71585,7,I,2022-10-28 00:00:00 +0000,38340375.03
7,NaN,NaN,C00797670,2022,N00038397,Maggie Hassan (D),American Israel Public Affairs Cmte,51650,10000,61650,8,I,2022-10-28 00:00:00 +0000,38340375.03
8,NaN,NaN,C00215046,2022,N00038397,Maggie Hassan (D),Fidelity Investments,49201,9000,58201,9,I,2022-10-28 00:00:00 +0000,38340375.03
9,NaN,D000000085,NaN,2022,N00038397,Maggie Hassan (D),Goldman Sachs,42650,10000,52650,10,I,2022-10-28 00:00:00 +0000,38340375.03


In [12]:
all_processed_contribs = pd.DataFrame()
for file in glob("raw_contributions/*"):
    contribs = pd.read_csv(file)
    race = file.split("/")[-1].split(".")[0]
    state = race[:2]
    processed_contribs = contribs.apply(extract_contrib, axis=1)
    processed_contribs = processed_contribs.assign(race=race)
    processed_contribs = processed_contribs.assign(state=state)
    all_processed_contribs = pd.concat([all_processed_contribs, processed_contribs])
all_processed_contribs

,firstname,lastname,party,amount,contributor,race,state
0,cory,gardner,republican,124600,Votesane PAC,COS2,CO
1,cory,gardner,republican,64961,Apollo Global Management,COS2,CO
2,cory,gardner,republican,55979,GEO Group,COS2,CO
3,cory,gardner,republican,51765,Blue Cross/Blue Shield,COS2,CO
4,cory,gardner,republican,50788,Blackstone Group,COS2,CO
...,...,...,...,...,...,...,...
38,don,bolduc,republican,2900,Nh Leadership Pac,NHS1,NH
39,don,bolduc,republican,2900,Nixon Peabody LLP,NHS1,NH
40,jeremy,kauffman,other,2200,Brennan Appraisal,NHS1,NH
41,jeremy,kauffman,other,1500,Gfy LLC,NHS1,NH


In [13]:
senators = pd.read_csv("S117_members.csv")

code_to_party = {100: "democrat", 200: "republican"}
def create_senator(senator):
    name = senator["bioname"]
    node_attrs = {
        "lastname": name.split(",")[0].lower(),
        "firstname": name.split(",")[1].split()[0].lower(),
        "party": code_to_party.get(senator["party_code"], "other"),
        "state": senator["state_abbrev"],
        "icpsr": senator["icpsr"]
    }
    return pd.Series(node_attrs)
senator_nodes = senators.apply(create_senator, axis=1)
senator_nodes

,lastname,firstname,party,state,icpsr
0,trump,donald,republican,USA,99912
1,biden,joseph,democrat,USA,99913
2,tuberville,thomas,republican,AL,42102
3,shelby,richard,republican,AL,94659
4,murkowski,lisa,republican,AK,40300
...,...,...,...,...,...
99,manchin,joe,democrat,WV,40915
100,baldwin,tammy,democrat,WI,29940
101,johnson,ron,republican,WI,41111
102,lummis,cynthia,republican,WY,20953


In [37]:
def find_icpsr(row):
    #senator = senator_nodes[senator_nodes["firstname"] == row["firstname"]]
    senator = senator_nodes[senator_nodes["lastname"] == row["lastname"]]
    senator = senator[senator_nodes["state"] == row["state"]]

    if len(senator) > 0:
        icpsr = senator.iloc[0].icpsr
    else:
        icpsr = -1
    return icpsr

all_processed_contribs["icpsr"] =  all_processed_contribs.apply(find_icpsr, axis=1)

<ipython-input-37-4d26eddf648c>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  senator = senator[senator_nodes["state"] == row["state"]]


In [38]:
all_processed_contribs[all_processed_contribs["icpsr"] != -1]

,firstname,lastname,party,amount,contributor,race,state,icpsr
20,john,hickenlooper,democrat,256481,Alphabet Inc,COS2,CO,42101
21,john,hickenlooper,democrat,238166,Democracy Engine,COS2,CO,42101
22,john,hickenlooper,democrat,176333,University of Colorado,COS2,CO,42101
23,john,hickenlooper,democrat,172769,University of California,COS2,CO,42101
24,john,hickenlooper,democrat,159910,J Street,COS2,CO,42101
...,...,...,...,...,...,...,...,...
15,maggie,hassan,democrat,42548,Massachusetts Institute of Technology,NHS1,NH,41702
16,maggie,hassan,democrat,41622,League of Conservation Voters,NHS1,NH,41702
17,maggie,hassan,democrat,40165,Dartmouth College,NHS1,NH,41702
18,maggie,hassan,democrat,38832,University of New Hampshire,NHS1,NH,41702


In [39]:
all_processed_contribs[all_processed_contribs["icpsr"] != -1].to_csv("contributions_with_icpsr.csv")